In [1]:
from IPython.display import clear_output

!pip install -U accelerate
!pip install -U transformers
!pip install datasets
!pip install evaluate

clear_output()

In [10]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import datasets
from datasets import load_dataset
import evaluate

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk

from collections import Counter
from typing import List
import string

import pandas as pd
import random
import os

import numpy as np
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
sns.set(palette='Paired')
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import re
import time
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")

In [82]:
df = pd.read_csv('/content/drive/MyDrive/train (4).csv',sep = '\t')

In [83]:
df = df.drop('idx',axis = 1)

In [84]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Score'] = le.fit_transform(df['Score'])

In [85]:
df

,Score,Text
0,1,В Альфа-Банке работает замечательная девушка -...
1,0,Оформляя рассрочку в м. Видео в меге тёплый ст...
2,1,Очень порадовала оперативность работы в банке....
3,0,Имела неосторожность оформить потреб. кредит в...
4,0,Небольшая предыстория: Нашел на сайте MDM банк...
...,...,...
13994,1,"О высокой надёжности МКБ, порядочности и добро..."
13995,1,"Обслуживаюсь в офисе на Чернореченской 42а, ка..."
13996,1,Попала сегодня в очень неприятную ситуацию. Ре...
13997,1,Добрый день! Давно являюсь клиентом банка Русс...


In [93]:
df2 = pd.DataFrame()
df2['text'] = df['Text']
df2['label'] = df['Score']

In [18]:
df2

,text,label
0,В Альфа-Банке работает замечательная девушка -...,1
1,Оформляя рассрочку в м. Видео в меге тёплый ст...,0
2,Очень порадовала оперативность работы в банке....,1
3,Имела неосторожность оформить потреб. кредит в...,0
4,Небольшая предыстория: Нашел на сайте MDM банк...,0
...,...,...
13994,"О высокой надёжности МКБ, порядочности и добро...",1
13995,"Обслуживаюсь в офисе на Чернореченской 42а, ка...",1
13996,Попала сегодня в очень неприятную ситуацию. Ре...,1
13997,Добрый день! Давно являюсь клиентом банка Русс...,1


In [19]:
df2.to_csv('train.csv',index = False)

In [94]:
# датасет для бинарной классификации
full_dataset_1 = load_dataset('csv', data_files='/content/train.csv')
# Фильтрация записей, в которых текстовые поля или метки не являются None
full_dataset_1 = full_dataset_1.filter(lambda example: example['text'] is not None and example['label'] is not None)

full_dataset_1

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 13999
    })
})

In [95]:
dataset = full_dataset_1['train'].train_test_split(test_size=0.2, seed=42)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 11199
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2800
    })
})

In [96]:
dataset['train'][0]

{'text': 'Пытался получить ипотечный кредит в период резкого роста цен на недвижимость. По неопытности метался между банками, побывал в 5-ти банках Петербурга. Райффайзен мне показался самым удобным.Плюсы:- вежливый внимательный персонал (я общался с тремя девушками в кредитном отделе и на ресепшн. Остались самые лучшие впечатления).- грамотно составленные анкеты (заполнил за 5 минут. Все просто, удобно, разумно).- никаких очередей (не считая телефонных. Каждый раз, когда я заходил в отделение на Ефимова или в отделение на Мойке, я быстро получал то, что мне было нужно).- хорошо проработанные процедуры (в качестве примера - порадовал недавний звонок: Вы так и не воспользовались кредитом, что предпочитаете - забрать документы или чтобы мы их уничтожили?, ни в одном другом не такого не предлагали).- конкурентноспособные банковские продукты (одна из самых низких ставок на ипотечный кредит у нас в городе).Минусы:- название. Произнести я еще это могу, но написать... Ни по-русски, ни латиниц

In [97]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import torch

model_name = "blanchefort/rubert-base-cased-sentiment-rusentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2,ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at blanchefort/rubert-base-cased-sentiment-rusentiment and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [98]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [99]:
# токенизация текста
dataset = dataset.map(lambda e:
                      tokenizer(e['text'],
                                truncation = True,
                                max_length=110,
                                padding='max_length'), batched=True)

Map:   0%|          | 0/11199 [00:00<?, ? examples/s]

Map:   0%|          | 0/2800 [00:00<?, ? examples/s]

In [100]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 11199
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2800
    })
})

In [101]:
dataset = dataset.remove_columns('text')

In [102]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 11199
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2800
    })
})

In [103]:
# переносим на гпу
dataset.set_format(type='torch')

In [104]:
dataset['train'][0]

{'label': tensor(1),
 'input_ids': tensor([  101, 52202,  3555, 16430,  1744,  7916,   340,  5904, 47275,  8590,
          2647,   801, 25993,   132,   914, 44429,  8030, 14785,   820,  1862,
         54820,   128, 31664,   340,   146,   130,  1481, 33957, 16090,   132,
         12822, 34435,   371, 13832,  1086, 35882,  5339, 48864,   132, 59941,
           156,   130, 63415, 84414, 27050,   120,   358, 26447,   336, 18891,
         43846,   340,  7916,  1256, 31693,   322,   801, 60358, 18179,   132,
         60790,  3495, 12021, 16910,   122,   132,   130, 21954, 32092,   906,
         65628,   120, 85521,   823,   146,  2512,   132,  1468,  1070,   128,
          4980,   128, 27546,   122,   132,   130,  3338, 52262,   120,   802,
         15774, 46102,   132,  6738,   893,   128,  1087,   358, 38014,   340,
         14939,   801, 64376, 20880,   997,   340, 14939,   801,  5443,   102]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [105]:
# тест модели
outputs = model(dataset['train']['input_ids'][0].unsqueeze(0).to(device), dataset['train']['attention_mask'][0].unsqueeze(0).to(device), dataset['train']['label'][0].unsqueeze(0).to(device))
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0757,  0.3562]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [108]:
from transformers import Trainer, TrainingArguments

# from datasets import load_metric
import evaluate

# Загрузка метрики вне функции
# metric_f1 = evaluate.load("accuracy")

# Параметры обучения
training_args = TrainingArguments(
        output_dir="./results",  # Каталог для сохранения результатов обучения
        num_train_epochs=4,  # Количество эпох обучения
        per_device_train_batch_size=32,  # Размер батча для обучения
        per_device_eval_batch_size=32,  # Размер батча для валидации
        learning_rate = 1e-4,
        # warmup_steps=500,  # Количество шагов разогрева
        # weight_decay=0.01,  # Сила L2 регуляризации
        # logging_dir="./runs",  # Каталог для логов TensorBoard
        # logging_steps=500,  # Логирование каждые 500 шагов
        evaluation_strategy="epoch",  # Стратегия оценки
        save_strategy="epoch",  # Стратегия сохранения модели
        load_best_model_at_end=True,  # Загрузка лучшей модели в конце
        metric_for_best_model="accuracy",  # Метрика для выбора лучшей модели
        # greater_is_better=True,  # Указывает, что большее значение метрики лучше
        remove_unused_columns=False,
        report_to="none")

# Функция для вычисления метрик
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {'accuracy': accuracy}

trainer = Trainer(
    model=model,  # Модель для обучения
    args=training_args,  # Аргументы обучения
    train_dataset=dataset["train"],  # Набор данных для обучения
    eval_dataset=dataset["test"],  # Набор данных для валидации
    compute_metrics=compute_metrics,  # Метрики для вычисления
)

In [109]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.521584,0.718929
2,0.488500,0.497330,0.777500
3,0.523100,0.437104,0.854286
4,0.523100,0.372388,0.860714


TrainOutput(global_step=1400, training_loss=0.5159043121337891, metrics={'train_runtime': 456.9014, 'train_samples_per_second': 98.043, 'train_steps_per_second': 3.064, 'total_flos': 2532217796776800.0, 'train_loss': 0.5159043121337891, 'epoch': 4.0})

In [55]:
test = pd.read_csv('/content/drive/MyDrive/test (3).csv',sep='\t')

In [56]:
test = test.drop('idx',axis = 1)

In [57]:
test

,Text
0,Замечательная история об открытии счета в банк...
1,"23.07.14г. я досрочно внес сумму, указанную оп..."
2,Была задолженность по кредитной карте. Банк по...
3,Срочно понадобилась определенная денежная сумм...
4,Здравствуйте. В местном отделении Бинбанка г. ...
...,...
5995,Уважаемые представители Русфинанс Банка! Прошу...
5996,04 июня сняли с дебетовой карты банка 10 000 T...
5997,В апреле решил я купить шуруповерт. Посмотрел ...
5998,Недавно взял потребкредит в этом банке. Все че...


In [75]:
smp = pd.read_csv('/content/sample_submission (5).csv',sep='\t')

In [76]:
smp

,idx,Score
0,13999,Positive
1,14000,Positive
2,14001,Positive
3,14002,Positive
4,14003,Positive
...,...,...
5995,19994,Positive
5996,19995,Positive
5997,19996,Positive
5998,19997,Positive


In [62]:
test

,Text
0,Замечательная история об открытии счета в банк...
1,"23.07.14г. я досрочно внес сумму, указанную оп..."
2,Была задолженность по кредитной карте. Банк по...
3,Срочно понадобилась определенная денежная сумм...
4,Здравствуйте. В местном отделении Бинбанка г. ...
...,...
5995,Уважаемые представители Русфинанс Банка! Прошу...
5996,04 июня сняли с дебетовой карты банка 10 000 T...
5997,В апреле решил я купить шуруповерт. Посмотрел ...
5998,Недавно взял потребкредит в этом банке. Все че...


In [65]:
test.to_csv('test.csv',index = False)

In [67]:
full_dataset_2 = load_dataset('csv', data_files='/content/test.csv')
# Фильтрация записей, в которых текстовые поля или метки не являются None
full_dataset_2 = full_dataset_1.filter(lambda example: example['Text'] is not None)

full_dataset_2

Filter:   0%|          | 0/6000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Text'],
        num_rows: 6000
    })
})

In [110]:
pred_dataset = full_dataset_2['train']

pred_dataset = pred_dataset.map(lambda e: tokenizer(e['Text'],
                                truncation = True,
                                max_length=110,
                                padding='max_length'), batched=True)
# Remove the raw text column to avoid issues with the model
pred_dataset = pred_dataset.remove_columns("Text")

# Initialize the Trainer
trainer = Trainer(model=model)

# Make predictions
predictions = trainer.predict(pred_dataset)

# Get predicted class indices
predicted_class_indices = predictions.predictions.argmax(-1)


Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [111]:
predicted_class_indices

array([1, 0, 0, ..., 0, 1, 0])

In [116]:
smp['Score'] = predicted_class_indices

In [117]:
smp

,idx,Score
0,13999,1
1,14000,0
2,14001,0
3,14002,0
4,14003,0
...,...,...
5995,19994,0
5996,19995,0
5997,19996,0
5998,19997,1


In [118]:
smp['Score'] = le.inverse_transform(smp['Score'])

In [119]:
smp

,idx,Score
0,13999,Positive
1,14000,Negative
2,14001,Negative
3,14002,Negative
4,14003,Negative
...,...,...
5995,19994,Negative
5996,19995,Negative
5997,19996,Negative
5998,19997,Positive


In [121]:
smp.to_csv('otv2.csv',sep='\t',index=False)